# --- Day 9: Movie Theater ---
https://adventofcode.com/2025/day/9

In [ ]:
from collections import defaultdict

def getTestInput():
	with open("sampleInput.txt") as file:
		return file.read()
	
def getInput():
	with open("redTiles.txt") as file:
		return file.read()

In [ ]:
redTiles = getInput().splitlines()

def getArea(corner1: str, corner2: str) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = [int(x) for x in corner1.split(",")]
	x2, y2 = [int(x) for x in corner2.split(",")]
	return abs(x1-x2+1) * abs(y1-y2+1)

# Loop through each corner pair
largestArea = 0
for i, tile1 in enumerate(redTiles):
	for tile2 in redTiles[i+1:]:
		area = getArea(tile1, tile2)
		largestArea = area if area > largestArea else largestArea

print(f"Largest rectangle area: {largestArea}")

# --- Part Two ---
My getArea function was broken ((abs(x1-x2)+1) * (abs(y1-y2)+1) != abs(x1-x2+1) * abs(y1-y2+1)) (Oops!)

So in a one week craze I implemented 3 solutions that all gave the wrong answer until I fixed my getArea function. Now that the function is fixed, they all work, but at very different speeds. I will outline each method.

### Coordinate Compression Method
This one is by far the fastest and is a method that I got from the subreddit. The idea is that you sort all of your x and y coordinates and map them to an index. Here's an example:

Lets say you have 4 points: 
- 100,150
- 50,200
- 400,20
- 10,20

Your x's sorted would be 10, 50, 100, 400

Your y's sorted would be 20, 150, 200

So your x compression map would be:
- 10 -> 1
- 50 -> 2
- 100 -> 3
- 400 -> 4

And your y compression map would be 
- 20 -> 1
- 150 -> 2
- 400 -> 3

So your new points are:
- 3,2
- 2,3
- 4,1
- 1,1

This makes a much smaller version of what is technically the same shape

From there you solve the small brain way: Get every point inside the shape, and for every rectangle you get check to make sure that every point overlaps with a point in the shape

This finishes running in about 5 seconds

In [ ]:
Tile = tuple[int, int] # Type alias for a tile
redTiles = getInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0])

# Creating coordinate compression and decompression maps
xCoords = sorted(list({x[0] for x in redTiles}))
yCoords =  sorted(list({x[1] for x in redTiles}))
xCompression = {num: compressedNum + 1 for compressedNum, num in enumerate(xCoords)}
yCompression = {num: compressedNum + 1 for compressedNum, num in enumerate(yCoords)}
xDecompression = {compressedNum + 1: num for compressedNum, num in enumerate(xCoords)}
yDecompression = {compressedNum + 1: num for compressedNum, num in enumerate(yCoords)}

def compressCoords(tile: Tile, xCompression: dict[int, int], yCompression: dict[int, int]) -> Tile:
	"""Returns compressed version of the coordinates"""
	return (xCompression[tile[0]], yCompression[tile[1]])

def getArea(tile1: Tile, tile2: Tile) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	return (abs(x1-x2)+1) * (abs(y1-y2)+1)

def getBorder(tile1: Tile, tile2: Tile) -> list[Tile]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return [(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)]
	return [(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)]

def floodFill(borders: list[Tile] | set[Tile], start: Tile, limit: int) -> set[Tile]:
	"""Returns all tiles that are inside the shape using a flood fill (dfs)"""
	stack = [start]
	explored = set(start)

	while stack:
		# If we've gone over our guess of how big the space will be return an empty set
		if len(explored) > limit:
			return set()
		
		# Get current tile and surrounding coordinates (successors)
		currentTile = stack.pop()
		surroundingCoords = [
			(currentTile[0] - 1, currentTile[1]),
			(currentTile[0] + 1, currentTile[1]),
			(currentTile[0], currentTile[1] - 1),
			(currentTile[0], currentTile[1] + 1)]
		
		# Loop through all surrounding coordinates and add them to the stack if they've not yet been seen
		for successor in surroundingCoords:
			if successor not in explored and successor not in borders:
				stack.append(successor)
				explored.add(successor)
	
	return explored

def getAllRedGreenTiles(borders: set[Tile], redTiles: set[Tile]) -> set[Tile]:
	# Start with over estimation of the size of the inside tiles
	limit = ((max({x[0] for x in compressedBorders}) - min({x[0] for x in compressedBorders}) + 1) *
			(max({x[1] for x in compressedBorders}) - min({x[1] for x in compressedBorders}) + 1))

	# Find any green tile (non corner border)
	greenTiles = list(borders - redTiles)

	# Loop through all green tiles
	for greenTile in greenTiles:
		# Get all points around it (potential start points)
		startTiles = [(greenTile[0] - 1, greenTile[1]),
			(greenTile[0] + 1, greenTile[1]),
			(greenTile[0], greenTile[1] - 1),
			(greenTile[0], greenTile[1] + 1)]

		# Loop through potential start points
		for start in startTiles:
			if start in greenTiles: continue
			# Skip if we're starting on a border
			if start in compressedBorders: continue
			# If we successfully complete the flood fill, return union of borders and new green tiles
			if greenTiles := floodFill(compressedBorders, start, limit):
				return borders | greenTiles
		
def getAllPairs(redTiles: set[Tile]) -> list[tuple[Tile, Tile]]:
	"""Gets every pair of red tiles"""
	pairs = []
	for i, tile1 in enumerate(redTiles):
		for tile2 in redTiles[i+1:]:
			pairs.append((tile1, tile2))

	return pairs

def getRectangleTiles(tile1: Tile, tile2: Tile) -> set[Tile]:
	"""Returns all tiles that are a part of the rectangle"""
	tiles = set()
	minX, maxX = min([tile1[0], tile2[0]]), max([tile1[0], tile2[0]])
	minY, maxY = min([tile1[1], tile2[1]]), max([tile1[1], tile2[1]])
	for x in range(minX, maxX + 1):
		for y in range(minY, maxY + 1):
			tiles.add((x,y))
			
	return tiles

def isValidRectangle(tile1: Tile, tile2: Tile, allRedGreenTiles: set[Tile]) -> bool:
	"""Returns False if any of the rectangle tiles appear outside of the shape.
	Otherwise it is a valid rectangle so return True"""
	minX, maxX = min([tile1[0], tile2[0]]), max([tile1[0], tile2[0]])
	minY, maxY = min([tile1[1], tile2[1]]), max([tile1[1], tile2[1]])
	for x in range(minX, maxX + 1):
		for y in range(minY, maxY + 1):
			if (x,y) not in allRedGreenTiles:
				return False
			
	return True

# Get all compressed borders
compressedBorders = set()
for i in range(len(redTiles) - 1):
	tile1 = compressCoords(redTiles[i], xCompression, yCompression)
	tile2 = compressCoords(redTiles[i + 1], xCompression, yCompression)
	compressedBorders |= set(getBorder(tile1, tile2))

# Loop through all pairs of compressed red tiles
compressedRedTiles = [compressCoords(tile, xCompression, yCompression) for tile in redTiles]
allRedGreenTiles = getAllRedGreenTiles(compressedBorders, set(compressedRedTiles))
largestValidRectangleArea = 0

for tile1, tile2 in getAllPairs(compressedRedTiles):
	# Decompress tiles before getting the area
	decompressedTile1 = compressCoords(tile1, xDecompression, yDecompression)
	decompressedTile2 = compressCoords(tile2, xDecompression, yDecompression)
	area = getArea(decompressedTile1, decompressedTile2)

	# If we've found a new largest valid area then update largestValidRectangleArea
	if area > largestValidRectangleArea and isValidRectangle(tile1, tile2, allRedGreenTiles):
		largestValidRectangleArea = area

print(f"Largest valid rectangle area: {largestValidRectangleArea}")

Largest valid rectangle area: 1540060480


### Shortest Sides Method

Second fastest solution and the second one that I came up with

First keep track of borders in a dictionary to be easily able to see all the borders on a specific axis. For instance you want to check all the borders on x=25 you can check by doing borderDict["X25"] which will return a list of Tiles of all borders on that axis

Then for each rectangle you get the two shortest parallel sides, and check for any borders inbetween those lines. This check is pretty fast since we're using a dictionary instead of looping through a list of all borders. If we find a border inbetween the two rectangles then there is a border inside the rectangle which means it's invalid (For this input. There is an edge case that invalidates this method)

This solution runs in about 50 minutes

In [204]:
Tile = tuple[int, int] # Type alias for a tile
#redTiles = getInput().splitlines()
redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0]) # Make sure we loop around

def getArea(tile1: Tile, tile2: Tile) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	return (abs(x1-x2)+1) * (abs(y1-y2)+1)

def getBorder(tile1: Tile, tile2: Tile) -> list[Tile]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return [(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)]
	return [(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)]

def getShortestSides(tile1: Tile, tile2: Tile) -> tuple[list[Tile], list[Tile]]:
	"""Returns the two shortest sides of a rectangle given opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	xRangeSmaller = abs(x1-x2) > abs(y1-y2)
	# Get the two smallest sides
	if xRangeSmaller:
		side1 = getBorder((x1, y1), (x1, y2))
		side2 = getBorder((x2, y1), (x2, y2))
	else:
		side1 = getBorder((x1, y1), (x2, y1))
		side2 = getBorder((x2, y2), (x1, y2))
	
	return side1, side2

def isValidRectangle(tile1: Tile, tile2: Tile, borderDict: dict[str, list[Tile]]) -> bool:
	"""Checks to see if there are any borders on the inside of the rectangle
	If there are then it returns Flase
	Otherwise return True"""
	side1, side2 = getShortestSides(tile1, tile2)

	for tile1, tile2 in list(zip(side1, side2))[1:-1]:
		# If X's are the same between sides, check columns to see if there 
		# are any borders in the area of the rectangle 
		if tile1[0] == tile2[0]:
			minY, maxY = min([tile1[1], tile2[1]]), max([tile1[1], tile2[1]])
			borderInArea = any([minY < tile[1] and tile[1] < maxY for tile in borderDict[f"X{tile1[0]}"]])
			if borderInArea:
				return False
		
		# If Y's are the same between sides, check rows to see if there 
		# are any borders in the area of the rectangle
		elif tile1[1] == tile2[1]:
			minX, maxX = min([tile1[0], tile2[0]]), max([tile1[0], tile2[0]])
			borderInArea = any([minX < tile[0] and tile[0] < maxX for tile in borderDict[f"Y{tile1[1]}"]])
			if borderInArea:
				return False
		else:
			print("Something has gone horribly wrong")
	
	return True

# I know this looks stupid but its used for quick lookups on a specific axis
borderDict = defaultdict(set)
for i in range(len(redTiles) - 1):
	for tile in getBorder(redTiles[i], redTiles[i + 1]):
		borderDict[f"X{tile[0]}"].add(tile)
		borderDict[f"Y{tile[1]}"].add(tile)

# Loop through each pair of red tiles, if it is a valid rectangle and 
# the are is the largest valid one we've seen yet then update largestValidArea 
largestValidArea = 0
for i, tile1 in enumerate(redTiles):
	for tile2 in redTiles[i+1:]:
		area = getArea(tile1, tile2)
		if area > largestValidArea and isValidRectangle(tile1, tile2, borderDict):
			largestValidArea = area

print(f"Largest valid rectangle area: {largestValidArea}")

Largest valid rectangle area: 24


### Outer Border Method
This is the first thing I tried and by far the slowest. The essential idea is that you trace the outer border around the shape, and when you check every rectangle, if any of the perimeter overlaps with the outer border, then it is invalid. This is what I mean by getting the outer border: 

Regular border:
```
............
............
..BBBBBBBB..
..B......B..
..B......B..
..BBBBBBBB..
............
............
```

Outer border:
```
............
.OOOOOOOOOO.
.OBBBBBBBBO.
.OB......BO.
.OB......BO.
.OBBBBBBBBO.
.OOOOOOOOOO.
............
```

I have not waited for the entire thing to finish all at once (I did skip to a point close to the end to make sure it works) but my guess is that it would take a max of 14 hours to finish running, although if I had to guess more like somewhere between 7 and 10 hours

In [ ]:
Tile = tuple[int, int] # Type alias for a tile
#redTiles = getInput().splitlines()
redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0]) # Make sure we loop back to the start
		
def getArea(tile1: Tile, tile2: Tile) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	return (abs(x1-x2)+1) * (abs(y1-y2)+1)

def getBorder(tile1: Tile, tile2: Tile) -> list[Tile]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return set([(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)])
	return set([(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)])

def getAdjacentTiles(tile: Tile) -> list[Tile]:
	"""Get all 8 adjacent tiles"""
	tileChanges = [[-1,-1], [-1,0], [-1,1], [0,-1], [0,1], [1,-1], [1,0], [1,1]]
	tileX, tileY = tile
	return [(tileX + xChange, tileY + yChange) for xChange, yChange in tileChanges]

def getTileScore(tile: Tile, borders: set[Tile]) -> int:
	"""Get all 8 adjacent tiles and return how many are part of the border"""	
	return len([x for x in getAdjacentTiles(tile) if x in borders])

def getStartTile(borders: set[Tile]) -> Tile:
	"""Get the first outer border tile by moving diagonally until it hits a border"""
	xy = 0
	while True:
		xy += 1
		if getTileScore((xy,xy), borders) > 0:
			return (xy, xy)

def getOuterBorder(borders: set[Tile]) -> set[Tile]:
	"""Find outer border using dfs"""
	startTile = getStartTile(borders)
	stack = [startTile]
	explored = set([startTile])

	# Continue looping while there are still tiles to be explored
	while stack:
		currentTile = stack.pop()
		
		# Get 8 surrounding coords, check to see if it is an outer border meaning:
		# It has not already been seen, it is not a border itself, and
		# it is connected to at least one other border
		for successor in getAdjacentTiles(currentTile):
			if successor not in explored and successor not in borders and getTileScore(successor, borders) > 0:
				stack.append(successor)
				explored.add(successor)
	
	return explored

def getAllPairs(redTiles: set[Tile]) -> list[tuple[Tile, Tile]]:
	"""Gets every pair of red tiles"""
	pairs = []
	for i, tile1 in enumerate(redTiles):
		for tile2 in redTiles[i+1:]:
			pairs.append((tile1, tile2))

	return pairs

def isValidRectangle(tile1: Tile, tile2: Tile, outerBorders: set[Tile]) -> bool:
	"""Check if there is any overlap between the perimeter of the rectangle and the outer borders"""
	tile3 = (tile1[0], tile2[1])
	tile4 = (tile2[0], tile1[1])
	# If at any point an intersection exists between the outer border and the rectangle perimeter
	# then it is an invalid rectangle 
	if outerBorders.intersection(getBorder(tile1, tile3)):
		return False
	if outerBorders.intersection(getBorder(tile1, tile4)):
		return False
	if outerBorders.intersection(getBorder(tile2, tile3)):
		return False
	if outerBorders.intersection(getBorder(tile2, tile4)):
		return False
	return True

# Get all borders
borders = set()
for i in range(len(redTiles) - 1):
	borders |= getBorder(redTiles[i], redTiles[i + 1])

# Get the outer borders from the border
outerBorders = getOuterBorder(borders)

# Loop through all pairs sorted by area
# The first valid rectangle will be the largest valid rectangle
allPairs = getAllPairs(redTiles)
allPairs.sort(reverse=True, key=lambda x: getArea(x[0], x[1]))
largestValidRectangleArea = 0
for tile1, tile2 in allPairs:
	if isValidRectangle(tile1, tile2, outerBorders):
		largestValidRectangleArea = getArea(tile1, tile2)
		break

print(f"Largest valid rectangle area: {largestValidRectangleArea}")

Largest valid rectangle area: 24
